In [53]:
from PIL import Image, ImageDraw
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import warnings
import os
from scipy.io import loadmat
warnings.filterwarnings("ignore")

In [54]:
# masks generated for binrushed
def get_mask(file_path):
    img = cv2.imread(file_path)
    img_grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(img_grey, 60, 200)
    contours, hierarchy = cv2.findContours(edged,cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    empty = np.zeros(img.shape[:2])
#     print(type(img_grey))
#     empty = Image.new('L', tuple(img_grey.shape[1::-1]))
    outer_ind = 0
    for i, cont in enumerate(contours):
        if cv2.contourArea(cont) > 2500 and cv2.contourArea(cont) < 20000:
            inner_ind = i
        if cv2.contourArea(cont) > cv2.contourArea(contours[outer_ind]):
            outer_ind = i
    
    cv2.drawContours(empty, contours, outer_ind, 1, cv2.FILLED) 
#     cv2.drawContours(empty, contours, outer_ind, 1, cv2.FILLED) 

    try:
        cv2.drawContours(empty, contours, inner_ind, 2, cv2.FILLED)
#         cv2.drawContours(empty, contours, inner_ind, 2, cv2.FILLED) 
        return empty
    except:
        print('No mask found!')
        return None
    

In [10]:
folder = r'C:\Users\sauma\Documents\Glaucoma Research\BinRushed\BinRushed1-Corrected'
mask_folder = r'C:\Users\sauma\Documents\Glaucoma Research\BinRushed\BinRushed1-Correct_Masks'
for img in os.listdir(folder):
    path = os.path.join(folder, img)
    mask = get_mask(path)
    if mask is not None:
        im = Image.fromarray(mask.astype('uint8'), 'L')
        output_path = os.path.join(mask_folder, img)[:-4] + '.png'
        im.save(output_path)
        

In [5]:
import json
# G1020 Masks
def json_mask(img_path, json_path):
    image = cv2.imread(img_path)
    plt.imshow(image)
    plt.show()
    empty_mask = Image.new('L', tuple(image.shape[1::-1]))
    with open(json_path) as f:
        data = json.load(f)
    shapes = data['shapes']
    for i, item in enumerate(data['shapes']):
        label = item['label']
        if label == 'disc':
            point_disc = item['points']
        elif label == 'cup':
            point_cup = item['points']
    print(point_cup)
    try:
        polygon=tuple(map(tuple, point_disc))
        ImageDraw.Draw(empty_mask).polygon(polygon, outline=1, fill=1)
    except:
        pass
    try:
        polygon = tuple(map(tuple, point_cup))
        ImageDraw.Draw(empty_mask).polygon(polygon, outline=2, fill=2)
    except:
        return empty_mask
    return empty_mask

In [12]:
folder = r'C:\Users\sauma\Documents\Glaucoma Research\G1020'
mask_folder = r'C:\Users\sauma\Documents\Glaucoma Research\G1020_Masks'
for file in os.listdir(folder):
    if file[-4:] == '.jpg':
        img_path = os.path.join(folder, file)
        json_path = os.path.join(folder, file[:-3]+'json')
        mask = json_mask(img_path, json_path)
#         mask.save(os.path.join(mask_folder, file[:-3]+'png'))
        

In [36]:
def drishti_mask(img_path, folder_path, name):
    cup_bound = os.path.join(os.path.join(folder_path, 'AvgBoundary'), f'{name}_CupAvgBoundary.txt')
    disc_bound = os.path.join(os.path.join(folder_path, 'AvgBoundary'), f'{name}_ODAvgBoundary.txt')
    image = cv2.imread(img_path)
    cup_points = pd.read_csv(cup_bound, sep=' ', header=None)
    disc_points = pd.read_csv(disc_bound, sep=' ', header=None)
    point_cup = []
    for y, x in zip(cup_points[0], cup_points[1]):
        point_cup.append([x, y])
    point_disc = []
    for y, x in zip(disc_points[0], disc_points[1]):
        point_disc.append([x, y])
        
    empty_mask = Image.new('L', tuple(image.shape[1::-1]))
    try:
        polygon=tuple(map(tuple, point_disc))
        ImageDraw.Draw(empty_mask).polygon(polygon, outline=1, fill=1)
    except:
        pass
    try:
        polygon = tuple(map(tuple, point_cup))
        ImageDraw.Draw(empty_mask).polygon(polygon, outline=2, fill=2)
    except:
        return empty_mask
    return empty_mask

In [38]:
img_path = r'C:\Users\sauma\Documents\Glaucoma Research\Drishti\data\DRISHTI_GS\Drishti-GS1_files\Training\Images'
folder_path = r'C:\Users\sauma\Documents\Glaucoma Research\Drishti\data\DRISHTI_GS\Drishti-GS1_files\Training\GT'
out_folder = r'C:\Users\sauma\Documents\Glaucoma Research\Drishti\data\DRISHTI_GS\Drishti-GS1_files\Training\Masks'
for img_name in os.listdir(folder_path):
    full_folder_path = os.path.join(folder_path, img_name)
    full_img_path = os.path.join(img_path, img_name+'.png')
    mask = drishti_mask(full_img_path, full_folder_path, img_name)
    mask.save(os.path.join(out_folder, img_name+'.png'))

In [46]:
def origa_mask(mat_file):
    mat = loadmat(mat_file)
    return mat['mask']

In [52]:
mask_folder = r'C:\Users\sauma\Documents\Glaucoma Research\ORIGA\Semi-automatic-annotations'
new_folder = r'C:\Users\sauma\Documents\Glaucoma Research\ORIGA\Masks'
for item in os.listdir(mask_folder):
    if item[-3:] == 'mat':
        mask = origa_mask(os.path.join(mask_folder, item))
        cv2.imwrite(os.path.join(new_folder, item[:-3]+'png'), mask)